In [5]:
import os
import pandas as pd

# Step 1: Define the file path
file_path = '../data/your_dataset.csv'  # Adjust this to the correct path relative to your current working directory

# Step 2: Check if the file exists at the specified path
if os.path.exists(file_path):
    # If file exists, print the absolute path and load the dataset
    print(f"✅ File found at: {os.path.abspath(file_path)}")
    df = pd.read_csv(file_path)  # Load the dataset into a pandas DataFrame
    print("✅ Dataset loaded successfully.")
else:
    # If the file is not found, raise a FileNotFoundError with the correct path
    print(f"❌ The dataset file was not found at the path: {os.path.abspath(file_path)}")
    raise FileNotFoundError(f"The dataset file was not found at the path: {file_path}")

# Step 3: Perform basic data quality checks on the loaded dataset
if 'df' in locals():  # Check if the DataFrame exists (i.e., the dataset was loaded successfully)
    # Print basic information about the dataset
    print("\n✅ Basic Data Overview:")
    print(df.info())  # Shows the summary of the dataset (columns, data types, non-null count)
    
    # Display the first few rows of the dataset to get a quick look at the data
    print("\n✅ First 5 Rows of the Dataset:")
    print(df.head())
    
    # Step 4: Check for missing values
    print("\n✅ Missing Values in Each Column:")
    print(df.isnull().sum())  # Check for missing values in each column
    
    # Step 5: Check for duplicate rows
    print("\n✅ Duplicates in the Dataset:")
    print(f"Total Duplicates: {df.duplicated().sum()}")  # Check if there are any duplicate rows
    df_no_duplicates = df.drop_duplicates()  # Remove duplicates if any
    print(f"✅ Duplicates Removed. New Shape: {df_no_duplicates.shape}")
    
    # Step 6: Basic Statistical Summary
    print("\n✅ Statistical Summary of the Dataset:")
    print(df.describe())  # Show statistics for numerical columns
else:
    print("❌ Dataset was not loaded successfully. Exiting the program.")


❌ The dataset file was not found at the path: /workspaces/AI_DATA_ANALYSIS_/src/Module 4/data/your_dataset.csv


FileNotFoundError: The dataset file was not found at the path: ../data/your_dataset.csv

In [ ]:
# Part 2: Automating Data Quality Checks
# Objective: Use Python and data quality frameworks to automate validation.

# Task 1: Setting Up Automated Validation with Python

# Task 2: Introduction to Great Expectations: Install the great_expectations package and set up a basic project.

# Task 3: Creating Expectations with Great Expectations: Use Great Expectations to define data validation expectations for a dataset.


